In [9]:
import os 
from dotenv import load_dotenv,find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [11]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./data/5pages.pdf"

loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))


4


### RAG

In [12]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits,embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()



Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [13]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you"
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm,prompt)

rag_chain = create_retrieval_chain(retriever,question_answer_chain)

results = rag_chain.invoke({"input":"what is this article about"})

results["answer"]

'This article provides instructions on creating PDF documents from scanned files or through an alternative method if Acrobat Standard or higher is not available. It includes details on scanner settings for document scanning and steps for converting files to PDF using PS2PSF if needed.'